In [3]:
import os
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    raise ValueError("❌ GEMINI_API_KEY not found in .env")

client = genai.Client(api_key=api_key)



client = genai.Client()
model_id = "gemini-2.5-flash-preview-05-20"

google_search_tool = Tool(
    google_search = GoogleSearch()
)

response = client.models.generate_content(
    model=model_id,
    contents="Latest guideline for HTN control for patients with T2DM.",
    config=GenerateContentConfig(
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)

for each in response.candidates[0].content.parts:
    print(each.text)
# Example response:
# The next total solar eclipse visible in the contiguous United States will be on ...

# To get grounding metadata as web content.
print(response.candidates[0].grounding_metadata.search_entry_point.rendered_content)

The latest guidelines for hypertension control in patients with Type 2 Diabetes Mellitus (T2DM) emphasize individualized blood pressure targets and comprehensive management strategies including lifestyle modifications and pharmacological interventions.

Key recommendations from major organizations include:

**Blood Pressure Targets:**
*   **European Society of Cardiology (ESC) 2024 Guidelines:** Recommend a target office systolic blood pressure (SBP) of 120–129 mmHg and diastolic blood pressure (DBP) of 70–79 mmHg for most patients with diabetes, if feasible and tolerated. For older patients (age ≥65 years), the target SBP should be 130–139 mmHg if tolerated. The DBP target for all treated patients is <80 mmHg. Pharmacological treatment is recommended for those with confirmed hypertension (office BP ≥140/90 mmHg) and for those with office BP ≥130/80 mmHg after a maximum of three months of lifestyle intervention to reduce cardiovascular disease (CVD) risk. An on-treatment SBP target of 

In [4]:
import os
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import json

# --- Load Gemini API Key ---
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    raise ValueError("❌ GEMINI_API_KEY not found in .env")

client = genai.Client(api_key=api_key)
model_id = "gemini-2.5-flash-preview-05-20"

google_search_tool = Tool(
    google_search = GoogleSearch()
)

# --- Query Gemini Model with Google Search Tool ---
response = client.models.generate_content(
    model=model_id,
    contents="Latest guideline for HTN control for patients with T2DM.",
    config=GenerateContentConfig(
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)

# Print Gemini answer (text)
for each in response.candidates[0].content.parts:
    print(each.text)

# --- Extract HTML grounding metadata ---
rendered_html = response.candidates[0].grounding_metadata.search_entry_point.rendered_content

# --- Parse HTML to extract references (titles/URLs) ---
soup = BeautifulSoup(rendered_html, "html.parser")
refs = [
    {"title": a.text.strip(), "url": a["href"]}
    for a in soup.find_all("a", class_="chip")
]

# --- Build LINE Flex Message JSON (for preview, or use with LINE SDK) ---
def references_to_flex(refs, headline="參考來源"):
    flex_contents = {
        "type": "bubble",
        "body": {
            "type": "box",
            "layout": "vertical",
            "contents": (
                [{"type": "text", "text": headline, "weight": "bold", "size": "lg", "margin": "md"}] +
                [
                    {
                        "type": "text",
                        "text": r["title"],
                        "size": "md",
                        "color": "#3366CC",
                        "action": {"type": "uri", "uri": r["url"]},
                        "margin": "md",
                        "wrap": True
                    }
                    for r in refs
                ]
            )
        }
    }
    return flex_contents

flex_msg_json = references_to_flex(refs)

# --- Preview Flex Message JSON ---
print("\n=== LINE Flex Message JSON ===\n")
print(json.dumps(flex_msg_json, ensure_ascii=False, indent=2))


The latest guidelines for hypertension (HTN) control in patients with Type 2 Diabetes Mellitus (T2DM) emphasize individualized care, intensive blood pressure (BP) targets, and a combination of lifestyle interventions and pharmacological treatment. Key recommendations from leading organizations include:

**Blood Pressure Targets:**
*   The **European Society of Cardiology (ESC) 2024 guidelines** recommend a target BP in clinical practice of 120–129/70–79 mmHg for most patients with T2DM and hypertension, if feasible and tolerated. For patients who cannot tolerate this target, a systolic BP level "as low as reasonably achievable" (ALARA principle) is recommended.
*   The **American Diabetes Association (ADA) 2023/2024 Standards of Care** advise that the target BP for most people with diabetes should be below 130/80 mmHg, if it can be safely achieved. For those with higher cardiovascular (CV) risk (e.g., established atherosclerotic cardiovascular disease or a 10-year ASCVD risk ≥15%), a t